In [1]:
import os
os.chdir('..')

In [2]:
from fox.config import Config
from fox.dataset import ComboDataset
from fox.model import Model
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl

In [3]:
c = Config(USE_PLANERCNN=False)
d = ComboDataset(c)
loader = torch.utils.data.DataLoader(d, batch_size=2, shuffle=True, collate_fn=d.collate_fn)
loader_val = torch.utils.data.DataLoader(ComboDataset(c, train=False), batch_size=2, collate_fn=d.collate_fn)

Caching labels data/mini/yolo/train.txt (64 found, 0 missing, 0 empty, 0 duplicate, for 64 images): 100%|██████████| 64/64 [00:00<00:00, 1265.82it/s]
Caching labels data/mini/yolo/test.txt (5 found, 0 missing, 0 empty, 0 duplicate, for 5 images): 100%|██████████| 5/5 [00:00<00:00, 821.25it/s]


In [4]:
model = Model(c, d.yolo_dataset.yolo_labels, len(loader))

Using cache found in /home/ouroboros/.cache/torch/hub/facebookresearch_WSL-Images_master
Loading weights:  weights/midas.pt
Model Summary: 147 layers, 5.93877e+07 parameters, 5.93877e+07 gradients


In [5]:
trainer = pl.Trainer(max_epochs=c.EPOCHS, gpus=1, automatic_optimization=False)
trainer.fit(
    model,
    train_dataloader=loader,
    val_dataloaders=loader_val
)

Epoch 0:  91%|█████████▏| 32/35 [02:22<00:13,  4.47s/it, loss=nan, v_num=93, total loss=6.02, midas loss=4.2e+3, yolo loss=1.82] 
Validating: 0it [00:00, ?it/s]
Epoch 1:  11%|█▏        | 4/35 [00:34<04:30,  8.72s/it, loss=nan, v_num=93, total loss=5.31, midas loss=3.43e+3, yolo loss=1.89, avg yolo val loss=85.6, yolo mAP=0, avg yolo loss=2.67, avg midas loss=3.52e+3, avg total loss=6.19]


1